In [1]:
import struct
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from math import exp
from sklearn.model_selection import train_test_split as split_
from random import seed
from sklearn import  metrics
import itertools

In [2]:
def read(fr):
    with open (fr,'rb')as f:
        zero,data_type,dims=struct.unpack('>HBB',f.read(4))
        shape=tuple(struct.unpack('>I',f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(),dtype=np.uint8).reshape(shape)

In [3]:
#this link to get mnist data set that you will use to train this model
#http://yann.lecun.com/exdb/mnist/

trean=read('train-images.idx3-ubyte')
data_trean=np.reshape(trean,(60000,28,28))
trean_l=read('train-labels.idx1-ubyte')
teast=read('t10k-images.idx3-ubyte')
data_teast=np.reshape(teast,(10000,28,28))
teast_l=read('t10k-labels.idx1-ubyte')
data_trean,r,trean_l,t = split_(data_trean,trean_l, test_size=5/6)
data_teast,r,teast_l,t = split_(data_teast,teast_l, test_size=0.9)
print(data_trean.shape)
print(data_teast.shape)

<ipython-input-2-e35aec660882>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(f.read(),dtype=np.uint8).reshape(shape)


(10000, 28, 28)
(1000, 28, 28)


In [4]:
def plot_con_mat(cm,classes,nomalize=False,title='confusion matrix',cmap=plt.cm.Blues):
    if nomalize:
        cm=cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
        print ('normalized confusion matrix')
    else:
        print('confusion matrix, without normalzion')
   
    plt.imshow(cm,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=45)
    plt.yticks(tick_marks,classes)
    fmt='.2f' if nomalize else 'd'
    thersh = cm.max()/2
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j,i,format(cm[i,j],fmt),
                horizontalalignment='center',
                color='white'if cm[i,j] > thersh else 'black')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
        

In [5]:
def reshape(df):
    df1=np.ones((len(df),16,7,7))
    for y in range(len(df)):
        d=-4
        for i in range(28):
            if i%7==0:
                d+=4
                e=0
            r=0
            for x in range(4):
                for w in range(7): 
                    df1[y][x+d][e][w]=df[y][i][w+r]
                r+=7
            e+=1
    return df1


In [6]:
def normalize(df):
    for y in range(len(df)):
        for i in range(28):
            for x in range(28):
                if df[y][i][x]>0:
                    df[y][i][x]=1

    return df

In [7]:
#data_trean=normalize(data_trean)
#data_teast=normalize(data_teast)
data_trean=reshape(data_trean)
data_teast=reshape(data_teast)

In [8]:
def fit(data_teast):
    f1=np.ones((len(data_teast),16,2))
    for e in range(len(data_teast)):
        for x in range(16):
            xc=0
            yc=0
            for i in range(7):
                if data_teast[e][x].sum()==0:
                    yc=4
                    xc=4
                    break
                yc+=data_teast[e][x,:,i].sum()*(i)
                xc+=data_teast[e][x][i].sum()*(i)
            if data_teast[e][x].sum()!=0:
                xc/=data_teast[e][x].sum()
                yc/=data_teast[e][x].sum()
            f1[e][x][1]=yc
            f1[e][x][0]=xc
    f1=np.reshape(f1,(len(data_teast),16*2))
    return f1


In [9]:
def indx(trean_l):
    t_l=[]
    for i in range(len(trean_l)):
        t_l.append(np.zeros(10))
        t_l[i][trean_l[i]]=1
    return t_l
t_l=indx(trean_l)
ts_l=indx(teast_l)



In [10]:
dtr1=fit(data_trean)
dts1=fit(data_teast)


In [11]:
def scal(data):
    for i in range(len(data)):
        for x in range(len(data[i])):
            data[i][x]/=3
    return data
dtr=scal(dtr1)
dts=scal(dts1)
    

In [12]:
class NN:
    def forwerd(self,img,w):
        #print(w.shape)
        sum1=np.dot(img,w)
        #print(sum1)
        o=1/(1+exp(-sum1))
        #print(o)
        #print('out')
        return o
    def layerout(self,img,w,nn):
        oj=[]
        for i in range(len(w)):
            oj.append(self.forwerd(self,img,w[i]))
        return oj
    def layers(self,img,w,nl,nn):
        allout=[]
        allout.append(img)
        for i in range(nl):
            allout.append(self.layerout(self,allout[i],w[i],nn))
        return allout
    def backout(self,oj,oi,t_l,w,eta):
        delta=np.ones(10)
        for i in range(10):
            #print(len(w[0]))
            delta[i]=t_l[i]-oj[i]
            #print(delta[i])
            delta[i]*=oj[i]
            #print(delta[i])
            delta[i]*=(1-oj[i])
            #print(delta[i])
            #print("sayed//////////////////////////////////////////////////////////////")
            for x in range(len(w)):
                #print(w[i][x])
                #print("//////////////////////////////////////////////////////////////")
                w[i][x]+=delta[i]*oi[x]*eta
                #print(w[i][x])
            #print(w)
        return delta,w      
    def data(self,data,nl,nn,t_l,eta,epochs): 
        w=[]
        np.random.seed(1)
        w.append(np.random.rand(nn,32)-.5)
        for i in range(nl-2):
            w.append(np.random.rand(nn,nn)-.5)
        w.append(np.random.rand(10,nn)-.5)
        for j in range(epochs):
            for x in range(len(data)):
                #print(w)
                oj=self.layers(self,data[x],w,nl,nn)
               # print(oj[-1])
                delta,w[-1]=self.backout(self,oj[-1],oj[-2],t_l[x],w[-1],eta)
                for i in range(-1,-nl,-1):
                   # print(i)
                    delta,w[i-1]=self.backhid(self,oj[i-1],oj[i-2],w[i],w[i-1],delta,eta)
                #print(delta)
                #print('sa')
        return w
    def fit(self,w,data,nl,nn):
        ind=[]
        for i in data:
            out=self.layers(self,i,w,nl,nn)[1:][-1]
            out1=sorted(out)
            #print(out.index(out1[-1]))
            ind.append(out.index(out1[-1]))
        return ind  
    def backhid(self,oj,oi,wkj,wji,delta_k,eta):
        delta=np.zeros(len(wji))
        for i in range(len(wji)): 
            #print(len(wji))
            #print(len(delta_k))
            delta[i]=(delta_k*wkj[:,i]).sum()
            #print(delta[i])
            delta[i]*=oj[i]
            #print(delta[i])
            delta[i]*=(1-oj[i])
            #print(delta[i])
            #print("sayed//////////////////////////////////////////////////////////////")
            for x in range(len(wji[i])):
                e=delta[i]*oi[x]
               # print(e)
                #print(len(wj[0]))
                #print("//////////////////////////////////////////////////////////////")
                wji[i][x]+=e*eta
                #print(w[i][x])
           #print(len(wj[i]))
        return delta,wji

In [ ]:
model=NN
nl=2
nn=300
e=50
w=model.data(model,dtr,nl,nn,t_l,1e-1,e)

In [ ]:
out=model.fit(model,w,dts,nl,nn)
from sklearn.metrics import accuracy_score
accuracy_score(out,teast_l)

In [ ]:
cm=metrics.confusion_matrix(teast_l,out)
plot_con_mat(cm,['0','1','2','3','4','5','6','7','8','9'],nomalize=True)